In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import pandas as pd

from multimodal_sft.utils import blend_sign, detect_shape, blend_polygon

sign_path = Path("../data/road_signs/")
train_path = Path("../data/train")

df = pd.read_csv("../data/shape.csv", header=None, names=["name", "shape", "color"])

In [ ]:
for j, path in enumerate((train_path/"images").glob("*.jpg")):
    img = cv2.imread(str(path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    with open(train_path/"labels"/(path.stem+".txt")) as f:
        for i, line in enumerate(f):
            annotation = list(map(float, line.split()))
            _, x, y, w, h = annotation
            x0, y0 = int(x * width), int(y * height)
            segment = np.load(train_path/"segmentations"/(path.stem+f"_{i}.npy"))
            h, w = segment.shape
            color = np.sum(img[y0:y0+h, x0:x0+w] * segment[..., np.newaxis], axis=(0, 1)) / np.sum(segment)
            dominant_color = "rgb"[color.argmax()]
            shape, area = detect_shape(segment.astype(np.uint8))
            if shape == "others":
                continue
            # 色と形が一致するサインを抽出
            sign_candidates = df[(df["shape"] == shape) & (df["color"] == dominant_color)]["name"].values
            if len(sign_candidates) == 0:
                continue
            # randomly select a sign
            sign_name = np.random.choice(sign_candidates)
            img_sign = cv2.imread(str(sign_path/f"{sign_name}.png"), -1)
            img_sign = cv2.cvtColor(img_sign, cv2.COLOR_BGRA2RGBA)
            img_sign_bin = (img_sign[:, :, 3] > 0).astype(np.uint8)
            _, area_sign = detect_shape(img_sign_bin)
            if shape == "circle":
                img = blend_sign(img, img_sign, annotation)
            else:
                img = blend_polygon(img, img_sign, area, area_sign, annotation)
            display(Image.fromarray(img))
    if j > 20:
        break